In [1]:
import json
import csv

from typing import List

from bs4 import BeautifulSoup
import requests
import tqdm

In [2]:
base_url = "https://bkdrf.ru"

In [3]:
url_map = "/map"
resp = requests.get(base_url + url_map)
soup = BeautifulSoup(resp.text, "lxml")

In [4]:
aglomerations = {}
html_aglomerations = soup.find(id="list-agglomeration").select("a")
for row in html_aglomerations:
    aglomeration = {
        "url": row["href"],
        "name": row.text        
    }
    row_id = row["data-agglomeration-id"]
    aglomerations[row_id] = aglomeration 

In [5]:
agl = aglomerations["tomsk"]
resp = requests.get(base_url + agl['url'])
soup = BeautifulSoup(resp.text, "lxml")

In [6]:
for script in soup.select("script"):
    if "var demoData" in script.decode():
        break
else:
    script = None

In [7]:
for line_data in script.decode().split("\n"):
    if "var demoData" in line_data:
        # fix encoding like this: '\\u0022title\\u0022:\\u0022Выполнение работ'
        line_data = line_data.split('"', 1)[1].rsplit('"',1)[0].replace("\\u0022", '"')
        # fix doublequotes escapes: '...\\\\"....'
        line_data = line_data.replace('\\\\"', '\\"')
        break
else:
    line_data = ""

In [8]:
tomsk_data = json.loads(line_data)

In [9]:
# tomsk_data["features"][0]
# type, geometry: {type=LineString|Point, coordinates: [[81.0, 58,9], ...]},
# properties: {id:uuid, status:int, worktypeid:uuid,title,victoryroad:bool}


In [10]:
for feature in tomsk_data.get("features", []):
    props = feature["properties"]
    print(feature["type"], feature["geometry"]["type"], props["status"], props["victoryroad"], props["title"])

Feature LineString 2 False Выполнение работ по ремонту и содержанию автомобильной дороги Могильный Мыс - Парабель - Каргасок на участке км 149+000 - км 157+360 в Каргасокском районе Томской области
Feature LineString 2 False Выполнение работ по ремонту автомобильной дороги Томск - Каргала - Колпашево на участке км 226+000 - км 249+000 в Чаинском районе Томской области
Feature LineString 2 False ул. Елизаровых от ул. Елизаровых, 11Б до ул. Шевченко, 55
Feature LineString 2 False ул. Артема от ул. Артема, 2 до ул. Артема, 22
Feature LineString 2 False ул. Усова от ул. Усова, 1 до ул. Усова, 9Б
Feature LineString 2 True ул. 79 Гв. Дивизии от ул. 79 Гв. Дивизии, 1 до ул. 79 Гв. Дивизии, 24
Feature LineString 2 True ул. 79 Гв. Дивизии от ул. 79 Гв. Дивизии, 1 до ул. 79 Гв. Дивизии, 24
Feature LineString 2 False ул. Д. Бедного от ул. Д. Бедного, 27 до ул. Д. Бедного, 28
Feature LineString 2 False пр. Ленина от пр. Ленина, 91 до пр. Ленина, 95
Feature LineString 2 False Иркутский тракт
Featur

In [11]:
len(tomsk_data["features"])

209

In [12]:
# load fragment

url_fragment = "/fragmentInfo"


In [13]:
# # parse fragment

# # header
# soup.find(name="p", attrs={"class": "mm_title"}).text.strip()
# # fragment_title
# p = soup.find(name="div", attrs={"class": "ob_main_info"}).find("p")
# if p.span:
#     p.span.extract()
# p.text.strip()
# props = soup.find("div", attrs={"class": "flex"}).find_all("p")
# for prop in props:
#     print(prop.span.text.strip(), ":\t", prop.span.next_sibling.strip())

In [14]:
def write_csv(file_name: str, collection: List[dict]):
    # list of columns (for CSV export of variable objects)
    csv_columns = list(set([k for elem in collection for k in elem.keys()]))
    
    with open(file_name, "wt+") as f:
        writer = csv.DictWriter(f, fieldnames=csv_columns)
        writer.writeheader()
        writer.writerows(collection)

In [15]:
def get_fragment(id_fragment) -> BeautifulSoup:
    resp = requests.get(base_url + url_fragment + "/" + id_fragment)
    return BeautifulSoup(resp.text, "lxml")

In [16]:
# list with object per fragment
collected_features = []

for feature in tqdm.tqdm(tomsk_data.get("features", [])):
    props = feature["properties"]
    id_fragment = props["id"]
    row = {
        "id_fragment": id_fragment,
        "type": feature.get("type"),
        "geometry_json": json.dumps(feature["geometry"]),
        "status": props.get("status"),
        "worktypeid": props.get("worktypeid"),
        "title": props.get("title"),
        "victoryroad": props.get("victoryroad"),
    }
    
    collected_features.append(row)

100%|██████████| 209/209 [00:00<00:00, 25685.18it/s]


In [17]:
fragments = set([f["id_fragment"] for f in collected_features])

collected_fragments = []

for id_fragment in tqdm.tqdm(fragments):
    row = {"id": id_fragment}
    
    fragment_data = get_fragment(id_fragment)

    row["fragment_header"] = fragment_data.find(name="p", attrs={"class": "mm_title"}).text.strip()

    row["fragment_title"] = fragment_data.find(name="div", attrs={"class": "ob_main_info"})\
                            .find("p").span.next_sibling.strip()

    fragment_props = fragment_data.find("div", attrs={"class": "flex"}).find_all("p")
    comments = []
    for prop in fragment_props:
        if prop.span:
            row[prop.span.text.strip()] = prop.span.next_sibling.strip()
        else:
            comments.append(prop.text.strip())
    if comments:
        row["comments"] = "\n".join(comments)
        
    collected_fragments.append(row)

100%|██████████| 166/166 [01:00<00:00,  2.73it/s]


In [18]:
write_csv("00_roads.csv", collected_features)
write_csv("00_fragments.csv", collected_fragments)

In [33]:
aglomerations

{'barnaul': {'url': '/map/barnaul', 'name': 'Алтайский край'},
 'blagoveshchensk': {'url': '/map/blagoveshchensk',
  'name': 'Амурская область'},
 'arhangelsk': {'url': '/map/arhangelsk', 'name': 'Архангельская область'},
 'astrahan': {'url': '/map/astrahan', 'name': 'Астраханская область'},
 'belgorod': {'url': '/map/belgorod', 'name': 'Белгородская область'},
 'bryansk': {'url': '/map/bryansk', 'name': 'Брянская область'},
 'vladimir': {'url': '/map/vladimir', 'name': 'Владимирская область'},
 'volgograd': {'url': '/map/volgograd', 'name': 'Волгоградская область'},
 'vologda': {'url': '/map/vologda', 'name': 'Вологодская область'},
 'voronezh': {'url': '/map/voronezh', 'name': 'Воронежская область'},
 'birobidzhan': {'url': '/map/birobidzhan',
  'name': 'Еврейская автономная область'},
 'chita': {'url': '/map/chita', 'name': 'Забайкальский край'},
 'ivanovo': {'url': '/map/ivanovo', 'name': 'Ивановская область'},
 'irkutsk': {'url': '/map/irkutsk', 'name': 'Иркутская область'},
 'nal